In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import sys
sys.path.insert(0, '../../../')

import numpy as np
import matplotlib.pyplot as plt
import pickle


from src.difsched.agents.mdp import MdpFormulator
from src.difsched.env import createEnv
from src.difsched.config import getSubAgentConfig, visualizeSubAgentConfig

In [ ]:
trafficDataParentPath = f'../../../data/processed/traffic'
for configIdx in [1]:
    subAgentParams = getSubAgentConfig(configIdx)
    visualizeSubAgentConfig(subAgentParams)
    env = createEnv(subAgentParams, trafficDataParentPath)
    env.selectMode(mode="train", type="data")

    mdpFormulator = MdpFormulator(subAgentParams, env.trafficGenerator.getM(mode="train"))
    mdpFormulator.aggregateModel(approximate=True)

    mdpKernel, mdpKernelParams = mdpFormulator.getMdpKernel()
    print(mdpKernel.rewardTable.shape)
    print(mdpKernel.transitionTable.shape)
    (V_deter, policy_deter) = mdpKernel.optimize_policy(mode="deterministic", max_iterations=1000)
    mdpKernelParams['V_deter'] = V_deter
    mdpKernelParams['policy_deter'] = policy_deter  
     #(V_stoch, policy_stoch) = mdpKernel.optimize_policy(mode="stochastic", lr=10, max_iterations=80)
    mdpKernelParams['V_stoch'] = None
    mdpKernelParams['policy_stoch'] = None
    #mdpKernelParams['V_stoch'] = V_stoch
    #mdpKernelParams['policy_stoch'] = policy_stoch

    with open(f'../../data/results/MdpPolicy/mdp_config{configIdx}.pkl', 'wb') as f:
        pickle.dump(mdpKernelParams, f)


Environment Configuration
Number of Users:        4
Window Length:          200
Dataflow:               thumb_bk
N_aggregation:          4
N_r:                    5
Resource Bar:           5
Bandwidth:              60
Sigmoid K List:         [0.3]
Sigmoid S List:         [10.0]
Random Seed:            999
(256, 625)
(256, 256, 625)


Value Iteration:   4%|▎         | 37/1000 [00:09<03:55,  4.09it/s, V=8.812205]


KeyboardInterrupt: 

: 